In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/bnf.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [6]:
bnf_1min.tail()

,datetime,open,high,low,close,volume
687621,2024-06-07 15:25:00,49800.4500,49812.7500,49792.7000,49806.2000,0
687622,2024-06-07 15:26:00,49807.2500,49811.7000,49798.7500,49807.5000,0
687623,2024-06-07 15:27:00,49810.8500,49823.8000,49797.6000,49817.3000,0
687624,2024-06-07 15:28:00,49816.2500,49827.6000,49805.8000,49816.0000,0
687625,2024-06-07 15:29:00,49820.3000,49825.0000,49805.2500,49806.6000,0


In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [8]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2018, 7, 25),
 datetime.date(2023, 2, 2),
 datetime.date(2018, 12, 18),
 datetime.date(2019, 4, 26),
 datetime.date(2019, 6, 26),
 datetime.date(2022, 4, 27),
 datetime.date(2020, 3, 20),
 datetime.date(2022, 2, 11),
 datetime.date(2017, 10, 18),
 datetime.date(2017, 2, 14),
 datetime.date(2018, 4, 3),
 datetime.date(2019, 8, 8),
 datetime.date(2023, 4, 10),
 datetime.date(2017, 5, 24),
 datetime.date(2020, 3, 5),
 datetime.date(2020, 12, 11),
 datetime.date(2023, 10, 4),
 datetime.date(2018, 9, 4),
 datetime.date(2023, 1, 4),
 datetime.date(2024, 3, 13),
 datetime.date(2017, 8, 8),
 datetime.date(2018, 10, 12),
 datetime.date(2019, 12, 18),
 datetime.date(2024, 1, 29),
 datetime.date(2022, 6, 14),
 datetime.date(2019, 1, 18),
 datetime.date(2021, 9, 7),
 datetime.date(2024, 5, 23),
 datetime.date(2018, 6, 7),
 datetime.date(2017, 3, 28),
 datetime.date(2020, 9, 21),
 datetime.date(2021, 12, 17),
 datetime.date(2023, 5, 11),
 datetime.date(2023, 5, 19),
 datetime.date(20

In [9]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [10]:
async def create_sell_signals(df):

    df["sell_signal"] = False

    df["sell_signal"] = (df["high"] < df["high"].shift(1)) & (
        df["high"].shift(2) < df["high"].shift(1)
    )

    return df

async def create_buy_signals(df):
    df["buy_signal"] = False

    df["buy_signal"] = (df["low"] > df["low"].shift(1)) & (
        df["low"].shift(2) > df["low"].shift(1)
    )

    return df

async def create_ma(df, period):
    df["close"] = pd.to_numeric(df["close"], errors="coerce")
    df["ma"] = df["close"].rolling(window=period).mean()
    return df

In [11]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0001
LEVERAGE_ = 5

In [25]:
async def positional(df):

    in_trade = False
    tradebook = []
    remark = ''
    points = 0
    entry_price = None
    exit_price = None
    initial_sl = None
    # stoploss = None
    is_trailing_active = False

    for i in range(3, len(df)):
        # SHORT Variation
    
        if (not in_trade) and (df.iloc[i]['low'] < df.iloc[i-1]['low']) and (df.iloc[i-1]['sell_signal'] == True):
            #Entry Triggered
            if df.iloc[i]['open'] < df.iloc[i-1]['low']:
                if df.iloc[i]['high'] >= df.iloc[i-1]['low']:
                    in_trade = True
                    entry_price = df.iloc[i-1]['low']
                    entry_time = df.iloc[i]['datetime']
                    signal_time = df.iloc[i-1]['datetime']
                    initial_sl = max(df.iloc[i-1]['high'], df.iloc[i-2]['high'], df.iloc[i-3]['high'])
                    # stoploss = max(df.iloc[i-1]['high'], df.iloc[i-2]['high'], df.iloc[i-3]['high'])

        if in_trade:
            trade_entry_price = entry_price
            trade_initial_sl = initial_sl
            trade_final_sl = initial_sl

            if (
                not is_trailing_active
                and df.iloc[i]['high'] < df.iloc[i]['ma']
            ):
                is_trailing_active = True
            
            if not is_trailing_active:
                if df.iloc[i]['open'] > trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if df.iloc[i]['close'] >= trade_initial_sl:
                            in_trade = False
                            points = trade_entry_price - trade_initial_sl
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        
                        in_trade = False
                        points = trade_entry_price - df.iloc[i]['open']
                        exit_price = df.iloc[i]['open']
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif df.iloc[i]['high'] >= trade_initial_sl:
                    # Initial SL Hit

                    in_trade = False
                    points = trade_entry_price - trade_initial_sl
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

            else:
                trade_final_sl = min(trade_initial_sl, df.iloc[i]['ma'])
                if df.iloc[i]['open'] > trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if df.iloc[i]['close'] >= trade_initial_sl:
                            in_trade = False
                            points = trade_entry_price - trade_initial_sl
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        
                        in_trade = False
                        points = trade_entry_price - df.iloc[i]['open']
                        exit_price = df.iloc[i]['open']
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif df.iloc[i]['high'] >= trade_initial_sl:
                    # Initial SL Hit

                    in_trade = False
                    points = trade_entry_price - trade_initial_sl
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif df.iloc[i]['close'] >= trade_final_sl:
                    # Initial SL Hit

                    in_trade = False
                    points = trade_entry_price - df.iloc[i]['close']
                    exit_price = df.iloc[i]['close']
                    exit_time = df.iloc[i]["datetime"]
                    remark = "TSL Hit"
    
        if points:
            slippage = SLIPPAGE_ * (entry_price + exit_price)
            final_points = points - slippage
            # sl_in_points = abs(entry_price - initial_sl)
            qty = PORTFOLIO_VALUE * LEVERAGE_ / entry_price
            trade = {
                'Signal Time': signal_time,
                'Trade Type': 'SHORT',
                'Entry Date': entry_time.date(),
                'Entry Time': entry_time.time(),
                'Entry Price': entry_price,
                'Initial SL': trade_initial_sl,
                'Final SL': trade_final_sl,
                'Exit Date': exit_time.date(),
                'Exit Time': exit_time.time(),
                'Exit Price': exit_price,
                'Points Captured': points,
                'After Costs': final_points,
                'Remarks': remark,
                'Qty': qty,
                'ROI%': (points * qty / PORTFOLIO_VALUE) * 100,
                'ROI% w cs': (final_points * qty / PORTFOLIO_VALUE) * 100,
                'Trade Year': entry_time.year,
            }
            # print(trade)
            tradebook.append(trade)
            points = 0
            in_trade = False
            remark = ''
            entry_price = None
            exit_price = None
            initial_sl = None
            # stoploss = None
            is_trailing_active = False

    for i in range(3, len(df)):
        # LONG Variation
        
        if (not in_trade) and (df.iloc[i]['high'] >= df.iloc[i-1]['high']) and (df.iloc[i-1]['buy_signal'] == True):
            #Entry Triggered
            if df.iloc[i]['open'] > df.iloc[i-1]['high']:
                if df.iloc[i]['low'] <= df.iloc[i-1]['high']:
                    in_trade = True
                    entry_price = df.iloc[i-1]['high']
                    entry_time = df.iloc[i]['datetime']
                    signal_time = df.iloc[i-1]['datetime']
                    initial_sl = min(df.iloc[i-1]['low'], df.iloc[i-2]['low'], df.iloc[i-3]['low'])
                    # stoploss = max(df.iloc[i-1]['low'], df.iloc[i-2]['low'], df.iloc[i-3]['low'])

        if in_trade:
            trade_entry_price = entry_price
            trade_initial_sl = initial_sl
            trade_final_sl = initial_sl

            if (
                not is_trailing_active
                and df.iloc[i]['low'] > df.iloc[i]['ma']
            ):
                is_trailing_active = True
            
            if not is_trailing_active:
                if df.iloc[i]['open'] < trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if df.iloc[i]['close'] <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        
                        in_trade = False
                        points = df.iloc[i]['open'] - trade_entry_price
                        exit_price = df.iloc[i]['open']
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif df.iloc[i]['low'] <= trade_initial_sl:
                    # Initial SL Hit

                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

            else:
                trade_final_sl = max(trade_initial_sl, df.iloc[i]['ma'])
                if df.iloc[i]['open'] < trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if df.iloc[i]['close'] <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        
                        in_trade = False
                        points = df.iloc[i]['open'] - trade_entry_price
                        exit_price = df.iloc[i]['open']
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif df.iloc[i]['low'] <= trade_initial_sl:
                    # Initial SL Hit

                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif df.iloc[i]['close'] <= trade_final_sl:
                    # Initial SL Hit

                    in_trade = False
                    points = df.iloc[i]['close'] - trade_entry_price
                    exit_price = df.iloc[i]['close']
                    exit_time = df.iloc[i]["datetime"]
                    remark = "TSL Hit"    

        if points:
            slippage = SLIPPAGE_ * (entry_price + exit_price)
            final_points = points - slippage
            # sl_in_points = abs(entry_price - initial_sl)
            qty = PORTFOLIO_VALUE * LEVERAGE_ / entry_price
            trade = {
                'Signal Time': signal_time,
                'Trade Type': 'LONG',
                'Entry Date': entry_time.date(),
                'Entry Time': entry_time.time(),
                'Entry Price': entry_price,
                'Initial SL': trade_initial_sl,
                'Final SL': trade_final_sl,
                'Exit Date': exit_time.date(),
                'Exit Time': exit_time.time(),
                'Exit Price': exit_price,
                'Points Captured': points,
                'After Costs': final_points,
                'Remarks': remark,
                'Qty': qty,
                'ROI%': (points * qty / PORTFOLIO_VALUE) * 100,
                'ROI% w cs': (final_points * qty / PORTFOLIO_VALUE) * 100,
                'Trade Year': entry_time.year,
            }
            # print(trade)
            tradebook.append(trade)
            points = 0
            in_trade = False
            remark = ''
            entry_price = None
            exit_price = None
            initial_sl = None
            # stoploss = None
            is_trailing_active = False

    return pd.DataFrame(tradebook)

In [13]:
# async def trade(spot_data, index, tf):

#     if index == "bnf":
#         SPREAD = 100
#         index_full = "BANKNIFTY"
#     elif index == "nifty":
#         SPREAD = 50
#         index_full = "NIFTY"

#     start_date = dt.date(2017, 1, 1)
#     end_date = dt.date(2024, 3, 31)

#     flag = False

#     current_date = start_date

#     final_tb = pd.DataFrame()
#     tradebook = []

#     while current_date <= end_date:
#         # print(current_date)

#         if current_date.weekday() >= 5:
#             print("Skipping Weekend ! ", current_date)
#             current_date += dt.timedelta(days=1)
#             continue
#         elif current_date not in list_of_traded_dates:
#             print("Date Avoided ! ", current_date)
#             current_date += dt.timedelta(days=1)
#             continue
#         # elif current_date in list_of_dates_to_avoid_bnf:
#         #     # print('Date Avoided ! ', current_date)
#         #     current_date += dt.timedelta(days=1)
#         #     continue
#         else:
#             print("Working", current_date)
#             spot = spot_data[spot_data["datetime"].dt.date == current_date]
#             spot_open = spot.iloc[0]["open"]
#             # print(spot_open)
#             atm = int(round(spot_open / SPREAD) * SPREAD)
#             # print(atm)

#             current_expiry = await get_expiry(current_date, index)
#             dte = current_expiry - current_date

#             ce_df = await fetch_option_data(
#                 index=index,
#                 start_date=current_date,
#                 end_date=current_date,
#                 start_time=dt.time(9, 15),
#                 end_time=dt.time(15, 30),
#                 expiry=current_expiry,
#                 strike=atm,
#                 asset_class="C",
#             )
#             if not isinstance(ce_df, pl.DataFrame):
#                 print('Data Not Found for Options')
#                 current_date += dt.timedelta(days=1)
#                 continue
#             # print(ce_df.head())
#             ce_df = rename_ohlc_columns(ce_df)
#             # ce_5min = resample(ce_df, "5m")
#             ce_df = resample(ce_df, tf)
#             # call_option = ce_5min.to_pandas()
#             call_option = ce_df.to_pandas()

#             pe_df = await fetch_option_data(
#                 index=index,
#                 start_date=current_date,
#                 end_date=current_date,
#                 start_time=dt.time(9, 15),
#                 end_time=dt.time(15, 30),
#                 expiry=current_expiry,
#                 strike=atm,
#                 asset_class="P",
#             )
#             if not isinstance(pe_df, pl.DataFrame):
#                 print('Data Not Found for Options')
#                 current_date += dt.timedelta(days=1)
#                 continue
#             # print(pe_df.head())
#             pe_df = rename_ohlc_columns(pe_df)
#             # pe_5min = resample(pe_df, "5m")
#             pe_df = resample(pe_df, tf)
#             # put_option = pe_5min.to_pandas()
#             put_option = pe_df.to_pandas()

#             # call_option['datetime'] = pd.to_datetime(call_option['datetime'])
#             # put_option['datetime'] = pd.to_datetime(put_option['datetime'])

#             ce_df = await create_sell_signals(call_option)
#             pe_df = await create_sell_signals(put_option)

#             # print(ce_df.head())
#             # print(pe_df.head())

#             ce_contract = await get_option_contract_name(
#                 symbol=index_full,
#                 strike=atm,
#                 expiry=current_expiry,
#                 opt_type='CE'
#             )

#             pe_contract = await get_option_contract_name(
#                 symbol=index_full,
#                 strike=atm,
#                 expiry=current_expiry,
#                 opt_type='PE'
#             )

#             ce_trades = await option_trading(ce_df, ce_contract, atm, current_expiry)
#             pe_trades = await option_trading(pe_df, pe_contract, atm, current_expiry)

#             final_tb = pd.concat([final_tb, ce_trades, pe_trades], ignore_index=True)

#         current_date += dt.timedelta(days=1)

#     return final_tb

In [14]:
async def trade_positional(spot_data, index, tf, offset, ma):

    if index == "bnf":
        SPREAD = 100
        index_full = "BANKNIFTY"
    elif index == "nifty":
        SPREAD = 50
        index_full = "NIFTY"

    start_date = dt.date(2017, 1, 1)
    end_date = dt.date(2024, 3, 31)

    flag = False

    current_date = start_date

    final_tb = pd.DataFrame()
    tradebook = []

    # while current_date <= end_date:
        # print(current_date)

    # if current_date.weekday() >= 5:
    #     print("Skipping Weekend ! ", current_date)
    #     current_date += dt.timedelta(days=1)
    #     continue
    # elif current_date not in list_of_traded_dates:
    #     print("Date Avoided ! ", current_date)
    #     current_date += dt.timedelta(days=1)
    #     continue
    # # elif current_date in list_of_dates_to_avoid_bnf:
    # #     # print('Date Avoided ! ', current_date)
    # #     current_date += dt.timedelta(days=1)
    # #     continue
    # else:
    #     print("Working", current_date)
    data = resample(spot_data, tf, offset)
    data_pandas = data.to_pandas()
    data_pandas = await create_buy_signals(data_pandas)
    data_pandas = await create_sell_signals(data_pandas)
    data_pandas = await create_ma(data_pandas, ma)
    # print(data.head(50))
    # break
    tb = await positional(data_pandas)
    final_tb = pd.concat([tb, final_tb])

        # current_date += dt.timedelta(days=1)

    return final_tb

In [15]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% w cs"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0]["ROI% w cs"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0]["ROI% w cs"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% w cs"].cumsum() - combined_df_sorted["ROI% w cs"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [28]:
tfs = ['15m', '30m', '60m', '1d']
for i in range(5, 81, 5):
    for tf in tfs:
        if tf == '30m' or tf == '60m':
            ofs = '15m'
        else:
            ofs = '0m'
        variation = f'TF : {tf} , TMA : {i}'
        print(variation)
        x = await trade_positional(pl.DataFrame(bnf_1min), 'bnf', tf, ofs, i)
        x['Entry Datetime'] = x.apply(lambda row: pd.to_datetime(f"{row['Entry Date']} {row['Entry Time']}"), axis=1)
        x = x.sort_values(by='Entry Datetime')
        stats = generate_stats(x, variation)
# x

TF : 15m , TMA : 5
-2.1484906347173505 , -51.27533416930685 , -0.041901055732239886
TF : 30m , TMA : 5
46.90416363707107 , -35.9291695958315 , 1.305461945396948
TF : 60m , TMA : 5
-15.557217926717726 , -121.67857951271488 , -0.1278550258313302
TF : 1d , TMA : 5
-91.62738918593134 , -221.79252093094613 , -0.4131220872612698
TF : 15m , TMA : 10
13.904824039053988 , -52.017285287985324 , 0.26731160540332255
TF : 30m , TMA : 10
74.72872282071658 , -59.29814461656528 , 1.2602202531618616
TF : 60m , TMA : 10
-113.22822156080419 , -204.40194457700642 , -0.5539488471849962
TF : 1d , TMA : 10
-41.43185958525342 , -196.62269113433234 , -0.21071758984799588
TF : 15m , TMA : 15
-12.405262883263873 , -50.88307122330944 , -0.24379941275205583
TF : 30m , TMA : 15
49.10383189536759 , -83.16140678392492 , 0.5904641803733814
TF : 60m , TMA : 15
-124.56560660191923 , -238.31814213762715 , -0.5226862104773518
TF : 1d , TMA : 15
-2.7933588799707323 , -193.33426303335455 , -0.014448338520776396
TF : 15m , T

In [29]:
x['Entry Datetime'] = x.apply(lambda row: pd.to_datetime(f"{row['Entry Date']} {row['Entry Time']}"), axis=1)
x = x.sort_values(by='Entry Datetime')

In [30]:
roi = x['ROI%'].sum()
roi_w_cs = x['ROI% w cs'].sum()
roi, roi_w_cs

(445.87649142332333, 439.1045147949971)

In [31]:
x = await trade_positional(pl.DataFrame(bnf_1min), 'bnf', '30m', '15m', 10)
x['Entry Datetime'] = x.apply(lambda row: pd.to_datetime(f"{row['Entry Date']} {row['Entry Time']}"), axis=1)
x = x.sort_values(by='Entry Datetime')
stats = generate_stats(x, variation)

74.72872282071658 , -59.29814461656528 , 1.2602202531618616


In [27]:
for a, b in stats.items():
    print(a)
    print(b.to_string())

1.2602202531618616
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio           Variation
2017      -3.4698           35  34.2857                2.1942             -1.2957     -12.3728      -0.2804  TF : 1d , TMA : 80
2018      12.5211           33  36.3636                3.7933             -1.5713     -17.1515       0.7300  TF : 1d , TMA : 80
2019      67.9400           41  31.7073                8.7426             -1.6326     -17.0856       3.9765  TF : 1d , TMA : 80
2020      23.4801           40  35.0000                7.3451             -3.0520     -27.5563       0.8521  TF : 1d , TMA : 80
2021      -9.1626           36  19.4444                5.8960             -1.7391     -28.6834      -0.3194  TF : 1d , TMA : 80
2022      -6.8914           41  21.9512                4.3950             -1.4515     -18.1594      -0.3795  TF : 1d , TMA : 80
2023     -10.3380           51  33.3333                2.1730             -1.3906    

In [24]:
x.tail()

,Signal Time,Trade Type,Entry Date,Entry Time,Entry Price,Initial SL,Final SL,Exit Date,Exit Time,Exit Price,Points Captured,After Costs,Remarks,Qty,ROI%,ROI% w cs,Trade Year,Entry Datetime
1822,2024-05-30 15:15:00,LONG,2024-05-31,09:15:00,48838.6500,48545.0000,48789.3950,2024-05-31,09:45:00,48719.5000,-119.1500,-128.9058,TSL Hit,102.3779,-1.2198,-1.3197,2024,2024-05-31 09:15:00
97,2024-05-31 09:45:00,SHORT,2024-05-31,10:15:00,48719.5000,49079.0500,48766.0900,2024-05-31,12:45:00,48836.9500,-117.4500,-127.2056,TSL Hit,102.6283,-1.2054,-1.3055,2024,2024-05-31 10:15:00
1823,2024-05-31 12:45:00,LONG,2024-05-31,13:15:00,48863.4000,48681.2500,50690.5550,2024-06-04,09:15:00,49104.6000,241.2000,231.4032,TSL Hit,102.3261,2.4681,2.3679,2024,2024-05-31 13:15:00
1824,2024-06-04 12:45:00,LONG,2024-06-04,13:15:00,47399.9000,46077.8500,49267.7750,2024-06-06,12:15:00,49165.5000,1765.6000,1755.9435,TSL Hit,105.4855,18.6245,18.5226,2024,2024-06-04 13:15:00
1825,2024-06-06 13:15:00,LONG,2024-06-06,13:45:00,49378.8500,48906.7500,49738.0650,2024-06-07,14:15:00,49735.4500,356.6000,346.6886,TSL Hit,101.2579,3.6109,3.5105,2024,2024-06-06 13:45:00


In [66]:
x['Slippage in Points'] = x.apply(lambda row: 0.25 / 100 * (row['Entry Price'] + row['Exit Price']), axis=1)

In [ ]:
x['Final Points'] = x.apply(lambda row: row['Points Captured'] - row['Slippage in Points'])
x['PnL w cs'] = x.apply(lambda row: row['Final Points'] * row['Qty'])
x['ROI% w cs'] = x.aply(lambda row: row['PnL w cs'] * 100 / 1000000)

In [ ]:
# x

In [ ]:
async def trade_spot(spot_data, index, tf):

    start_date = dt.date(2017, 1, 1)
    end_date = dt.date(2024, 3, 31)

    flag = False

    current_date = start_date

    final_tb = pd.DataFrame()
    tradebook = []

    while current_date <= end_date:
        # print(current_date)

        if current_date.weekday() >= 5:
            print("Skipping Weekend ! ", current_date)
            current_date += dt.timedelta(days=1)
            continue
        elif current_date not in list_of_traded_dates:
            print("Date Avoided ! ", current_date)
            current_date += dt.timedelta(days=1)
            continue
        # elif current_date in list_of_dates_to_avoid_bnf:
        #     # print('Date Avoided ! ', current_date)
        #     current_date += dt.timedelta(days=1)
        #     continue
        else:
            print("Working", current_date)
            spot = spot_data[spot_data["datetime"].dt.date == current_date]
            spot_open = spot.iloc[0]["open"]
            # print(spot_open)

            spot_df = await create_sell_signals(spot_data)
            ce_df = await create_sell_signals(call_option)
            pe_df = await create_sell_signals(put_option)

            # print(ce_df.head())
            # print(pe_df.head())

            ce_contract = await get_option_contract_name(
                symbol=index_full,
                strike=atm,
                expiry=current_expiry,
                opt_type='CE'
            )

            pe_contract = await get_option_contract_name(
                symbol=index_full,
                strike=atm,
                expiry=current_expiry,
                opt_type='PE'
            )

            ce_trades = await option_trading(ce_df, ce_contract, atm, current_expiry)
            pe_trades = await option_trading(pe_df, pe_contract, atm, current_expiry)

            final_tb = pd.concat([final_tb, ce_trades, pe_trades], ignore_index=True)

        current_date += dt.timedelta(days=1)

    return final_tb